In [18]:
from flask import Flask, render_template
from markupsafe import escape

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text
from sqlalchemy.sql import func
import os

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(app.root_path, 'movie.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False 
app.app_context().push()

db = SQLAlchemy(app)

# many to many relation
movie_actor_association = db.Table(
    'movie_actor_association',
    db.Column('movie_id', db.Integer, db.ForeignKey('movie_info.movie_id')),
    db.Column('actor_id', db.Integer, db.ForeignKey('actor_info.actor_id')),
)

movie_director_association = db.Table(
    'movie_director_association',
    db.Column('movie_id', db.Integer, db.ForeignKey('movie_info.movie_id')),
    db.Column('actor_id', db.Integer, db.ForeignKey('actor_info.actor_id')),
)

class Movie(db.Model):
    __tablename__ = 'movie_info'

    movie_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    movie_name = db.Column(db.String(20))
    release_date = db.Column(db.String(15))
    country = db.Column(db.String(20))
    movie_type = db.Column(db.String(10))
    release_year = db.Column(db.Integer)
    
    moviebox = db.relationship('MovieBox', back_populates='movies', uselist=False)
    actors = db.relationship('Actor', secondary=movie_actor_association, back_populates='movies')
    director = db.relationship('Actor', secondary=movie_director_association, back_populates='movies')
    
    def __repr__(self):
        return f'<Movie {self.movie_name}>'
    
class Actor(db.Model):
    __tablename__ = 'actor_info'

    actor_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    actor_name = db.Column(db.String(20))
    gender = db.Column(db.String(2))
    country = db.Column(db.String(20))
    
    movies = db.relationship('Movie', secondary=movie_actor_association, back_populates='actors')
    
    def __repr__(self):
        return f'<Actor {self.actor_name}>'    
    
# one to one relation
class MovieBox(db.Model):
    __tablename__ = 'movie_box'
    movie_id = db.Column(db.String(10), db.ForeignKey('movie_info.movie_id'), unique=True, nullable=False, primary_key=True)
    movie_box = db.Column(db.Float)
    movies = db.relationship('Movie', back_populates='moviebox')

movies = Movie.query.all()

In [27]:
movies[-2].actors

[<Actor 吴京>, <Actor 易烊千玺>]

In [28]:
movies[-2].movie_id

1017

In [29]:
movie = Movie.query.get_or_404(1017)

if movie.moviebox:
    db.session.delete(movie.moviebox)
db.session.delete(movie)
db.session.commit()

NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

In [30]:
db.session.rollback()

In [31]:
movies

[<Movie 战狼2>,
 <Movie 哪吒之魔童降世>,
 <Movie 流浪地球>,
 <Movie 复仇者联盟4>,
 <Movie 红海行动>,
 <Movie 唐人街探案2>,
 <Movie 我不是药神>,
 <Movie 中国机长>,
 <Movie 速度与激情8>,
 <Movie 西虹市首富>,
 <Movie 复仇者联盟3>,
 <Movie 捉妖记2>,
 <Movie 八佰>,
 <Movie 姜子牙>,
 <Movie 我和我的家乡>,
 <Movie 你好，李焕英>,
 <Movie 长津湖>,
 <Movie 速度与激情9>]

In [ ]:
def delete(movie_id):
    movie = Movie.query.get_or_404(movie_id)
    
    # 解除与 MovieBox 的关联关系
    if movie.moviebox:
        db.session.delete(movie.moviebox)
    
    db.session.delete(movie)
    db.session.commit()
    
    flash('Item deleted.')
    return redirect(url_for('index'))